In [1]:
# import library
import time
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [2]:
# import cleaned data
df = joblib.load('cleaned_data.sav')

In [3]:
# visualization
df

,index,Id,Tags,Body,java,c#,javascript,python,jquery,html,...,regex,iphone,bash,objective-c,ios,windows,ruby,eclipse,tsql,database
0,0,4,[c#],convert decim doubl track bar form opac lang p...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,9,"[c#, .net]",calcul someon age base datetim birthday given ...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,11,[c#],calcul relat given specif datetim display rela...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,16,[c#],fill dataset datat linq queri result expo linq...,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,17,"[mysql, database]",binari mysql store binari http en wikipedia wi...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41163,41163,8057565,[mysql],mysql updat inner join queri idea problem mysq...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41164,41164,8054165,[html],put form put form send form,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41165,41165,8064691,"[javascript, html]",pas along variabl xmlhttprequest send variabl ...,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41166,41166,8057607,[java],mock privat powermock still mock privat jndi u...,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Target & Features

In [4]:
# creating feature and target variables
X = df.Body
Y = df.drop(columns = ['Id','index','Body','Tags'])

# Unsupervided method

## LDA

In [5]:
# topic extraction with LDA
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_fited = tfidf_vec.fit_transform(X)

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10,max_iter=5,learning_method='online',learning_offset=50,random_state = 0).fit(X_fited)

In [6]:
# displaying the 10 most frequent topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 2
display_topics(lda, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
string charact
Topic 1:
http java
Topic 2:
date datetim
Topic 3:
button input
Topic 4:
command directori
Topic 5:
git url
Topic 6:
list item
Topic 7:
android cs
Topic 8:
sql column
Topic 9:
java void


## NMF

In [7]:
# topic extraction with NMF
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit(X_fited)

NMF(alpha=0.1, init='nndsvd', l1_ratio=0.5, n_components=10, random_state=1)

In [8]:
# displaying the most 10 most frequent topic
no_top_words = 2
display_topics(nmf, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
http variabl
Topic 1:
string convert
Topic 2:
android activ
Topic 3:
java lang
Topic 4:
list item
Topic 5:
sql column
Topic 6:
git commit
Topic 7:
date datetim
Topic 8:
button input
Topic 9:
cs width


# Supervised method

## Bag of word

In [9]:
from gensim.corpora import Dictionary
body = df.Body

In [10]:
def Tokenization_texte(texte):
    x = []
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    for row in texte:
        x.append(tokenizer.tokenize(row))
    return x

In [11]:
tkz_X = Tokenization_texte(body)
dct = Dictionary(tkz_X)
df['Body']=tkz_X
X = df['Body'].apply(dct.doc2bow)

In [12]:
from gensim.matutils import corpus2csc
X = corpus2csc(X, num_terms=len(dct)).transpose()

## Train_test_split

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

## TF-IDF Vectorization

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_vec = TfidfTransformer()
X_train = tfidf_vec.fit_transform(X_train)
X_test = tfidf_vec.transform(X_test)

## Scores board

In [15]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# list of unique tags
categories = list(Y.columns.values)

# list of metrics used to compare model performance
scores = ['f1','accuracy','precision','recall','fitting_duration']


# scores dataframe for MultioutputClassifier
LinearSVC_scores_moc = pd.DataFrame(columns=scores)
MultinomialNB_scores_moc = pd.DataFrame(columns=scores)
Logistic_regression_scores_moc = pd.DataFrame(columns=scores)

# scores dataframe for chain classifier
LinearSVC_scores_ch = pd.DataFrame(columns=scores)
MultinomialNB_scores_ch = pd.DataFrame(columns=scores)
Logistic_regression_scores_ch = pd.DataFrame(columns=scores)

## Classifier Chain

### LinearSVC

In [16]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import ClassifierChain
base_clf = LinearSVC(random_state=0, tol=1e-5)
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [17]:
prediction = chain.predict(X_test)

In [18]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
LinearSVC_scores_ch['f1']=f1
LinearSVC_scores_ch['accuracy']=accuracy
LinearSVC_scores_ch['precision']=precision
LinearSVC_scores_ch['recall'] = recall
LinearSVC_scores_ch['fitting_duration'] = end-start

In [19]:
LinearSVC_scores_ch

,f1,accuracy,precision,recall,fitting_duration
0,0.600272,0.470779,0.693068,0.565544,2.602069


### MultinomialNB

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import ClassifierChain
base_clf = MultinomialNB()
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [21]:
prediction = chain.predict(X_test)

In [22]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
MultinomialNB_scores_ch['f1']=f1
MultinomialNB_scores_ch['accuracy']=accuracy
MultinomialNB_scores_ch['precision']=precision
MultinomialNB_scores_ch['recall'] = recall
MultinomialNB_scores_ch['fitting_duration'] = end-start

/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
MultinomialNB_scores_ch

,f1,accuracy,precision,recall,fitting_duration
0,0.033533,0.021125,0.443521,0.018198,0.596478


### Logistic regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
base_clf = LogisticRegression(max_iter=500,tol=1e-5)
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [25]:
prediction = chain.predict(X_test)

In [26]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
Logistic_regression_scores_ch['f1']=f1
Logistic_regression_scores_ch['accuracy']=accuracy
Logistic_regression_scores_ch['precision']=precision
Logistic_regression_scores_ch['recall'] = recall
Logistic_regression_scores_ch['fitting_duration'] = end-start

In [27]:
Logistic_regression_scores_ch

,f1,accuracy,precision,recall,fitting_duration
0,0.494619,0.394009,0.722519,0.4435,27.611098


## MultiOutputClassifier

### LinearSVC

In [28]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
clf = MultiOutputClassifier(LinearSVC(random_state=0, tol=1e-5),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [29]:
prediction = clf.predict(X_test)

In [30]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
LinearSVC_scores_moc['f1']=f1
LinearSVC_scores_moc['accuracy']=accuracy
LinearSVC_scores_moc['precision']=precision
LinearSVC_scores_moc['recall'] = recall
LinearSVC_scores_moc['fitting_duration'] = end-start

In [31]:
LinearSVC_scores_moc

,f1,accuracy,precision,recall,fitting_duration
0,0.614877,0.421242,0.780547,0.520268,1.621302


### MultinomialNB

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
clf = MultiOutputClassifier(LinearSVC(random_state=0, tol=1e-5),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [33]:
prediction = clf.predict(X_test)

In [34]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
MultinomialNB_scores_moc['f1']=f1
MultinomialNB_scores_moc['accuracy']=accuracy
MultinomialNB_scores_moc['precision']=precision
MultinomialNB_scores_moc['recall'] = recall
MultinomialNB_scores_moc['fitting_duration'] = end-start

In [35]:
MultinomialNB_scores_moc

,f1,accuracy,precision,recall,fitting_duration
0,0.023963,0.015015,0.395937,0.012804,0.291657


### Logistic regression

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
clf = MultiOutputClassifier(LogisticRegression(max_iter=500,tol=1e-5),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [37]:
prediction = clf.predict(X_test)

In [38]:
f1 = []
accuracy = []
precision = []
recall = []
f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
Logistic_regression_scores_moc['f1']=f1
Logistic_regression_scores_moc['accuracy']=accuracy
Logistic_regression_scores_moc['precision']=precision
Logistic_regression_scores_moc['recall'] = recall
Logistic_regression_scores_moc['fitting_duration'] = end-start

In [39]:
Logistic_regression_scores_moc

,f1,accuracy,precision,recall,fitting_duration
0,0.490646,0.313484,0.812579,0.372998,7.38452
